In [1]:
# https://sites.google.com/site/nttrungmtwiki/home/it/machine-lear/neural-networks/develop-your-first-neural-network-in-python-with-keras-step-by-step
from __future__ import print_function
import librosa
import numpy as np
import os
import sys
import argparse

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K

from mirex2016_dataset import load_trainXY


Using TensorFlow backend.


In [2]:
#datadir = './dataset/mirex_beat_tracking_2016/train/x862x1025_y40x862/'
datadir = './x862x128_y862x1/'


In [3]:
trainXY_path = datadir
#tb_logdir = args.logdir
epochs = 500
learning_rates = [10, 1, 0.1, 0.01, 0.006, 0.003, 0.001, 0.0006, 0.0003, 0.0001, 0.00005, 0.00001]
loss_function = 'binary_crossentropy'

In [4]:
trainX, trainY, n_of_frames, n_of_freq_bins = load_trainXY(trainXY_path)
#trainX.resize(n_of_frames, 1, n_of_freq_bins)
#trainY.resize(n_of_frames, 1, 1)
trainX.resize(trainX.shape[0], 1, n_of_freq_bins)
trainY.resize(trainY.shape[0], 1, 1)
#trainX.resize(n_of_frames*20*2, 1, n_of_freq_bins)
#trainY.resize(n_of_frames*20*2, 1, 1)


Load trainX.npy and trainY.npy (21, 128)


In [5]:
trainXP = trainX.reshape(trainX.shape[0],n_of_freq_bins)
trainYP = trainY.reshape(trainY.shape[0],1)
print('trainXP.shape: ' + str(trainXP.shape))
print('trainYP.shape: ' + str(trainYP.shape))



trainXP.shape: (17240, 128)
trainYP.shape: (17240, 1)


In [6]:
import pandas as pd
trainXPD  = np.diff(trainXP,axis=0)
trainYPD  = trainYP[1:]
trainXPDH = np.maximum(np.zeros(trainXPD.shape),trainXPD)
print(trainXPD.shape)
xx = pd.DataFrame(trainXP[0:4][:])
dd = pd.DataFrame(trainXPD[0:3][:])
hh = pd.DataFrame(trainXPDH[0:5][:])
display(xx)
print('\n')
display(dd)
print('\n')
display(hh)

(17239, 128)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
1,-78.416825,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
2,-16.782757,-12.069625,-7.636879,-10.055699,-12.186193,-14.297897,-21.732388,-26.662192,-26.433747,-27.860949,...,-55.524676,-55.514602,-53.402921,-55.703926,-58.417543,-63.255391,-72.216577,-80.0,-80.0,-80.0
3,-3.696009,-6.775470,-12.168939,-16.854252,-18.960096,-24.425571,-27.194247,-27.391886,-28.151292,-35.199834,...,-37.081288,-42.817498,-44.738287,-43.906941,-43.328234,-54.360452,-54.851924,-80.0,-80.0,-80.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,-4.532059,-6.798553,-6.773904,-10.127674,-5.461859,-0.729694,-1.717545,-7.338885,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0
3,0.000000,0.000000,0.739930,0.000000,0.000000,0.000000,0.000000,0.000000,1.397148,5.312010,...,0.000000,1.089796,0.000000,1.769560,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.804787,...,0.000000,1.297082,4.546216,0.000000,0.000000,26.797198,0.000000,0.0,0.0,0.0


In [7]:
def f1_score(y_true, y_pred):
    #Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    #If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
         return 0
    # How many selected items are relevant?
    precision = c1 / c2
    recall = c1 / c3
    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def recall(y_true, y_pred):
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0
    recall = c1 / c3
    return recall

In [8]:

lr = 0.0000025
X = trainXPDH
Y = trainYPD
# create model
model = Sequential()
model.add(Dense(10, input_dim=n_of_freq_bins, init='uniform', activation='relu'))
model.add(Dense(2, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='relu'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',recall])
# Fit the model
model.fit(X, Y, nb_epoch=500, batch_size=150)
# evaluate the model
scores = model.evaluate(X, Y,verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_initializer="uniform", input_dim=128, activation="relu")`
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="uniform", activation="relu")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="uniform", activation="relu")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/500
17239/17239 [==============================] - 0s - loss: 1.5862 - acc: 0.4863 - recall: 0.0975     
Epoch 2/500
17239/17239 [==============================] - 0s - loss: 1.1578 - acc: 0.5076 - recall: 0.2084     
Epoch 3/500
17239/17239 [==============================] - 0s - loss: 1.1115 - acc: 0.5057 - recall: 0.1601     
Epoch 4/500
17239/17239 [==============================] - 0s - loss: 1.1007 - acc: 0.5086 - recall: 0.1864     
Epoch 5/500
17239/17239 [==============================] - 0s - loss: 1.0759 - acc: 0.5112 - recall: 0.2144     
Epoch 6/500
17239/17239 [==============================] - 0s - loss: 1.0502 - acc: 0.5155 - recall: 0.2296     
Epoch 7/500
17239/17239 [==============================] - 0s - loss: 1.0530 - acc: 0.5146 - recall: 0.2523     
Epoch 8/500
17239/17239 [==============================] - 0s - loss: 1.0454 - acc: 0.5148 - recall: 0.2570     
Epoch 9/500
17239/17239 [==============================] - 0s - loss: 0.9912 - acc: 0.5157 - rec

17239/17239 [==============================] - 0s - loss: 0.6927 - acc: 0.5528 - recall: 0.4502     
Epoch 73/500
17239/17239 [==============================] - 0s - loss: 0.6911 - acc: 0.5609 - recall: 0.5595     
Epoch 74/500
17239/17239 [==============================] - 0s - loss: 0.6931 - acc: 0.5583 - recall: 0.5732     
Epoch 75/500
17239/17239 [==============================] - 0s - loss: 0.6920 - acc: 0.5075 - recall: 0.3084     
Epoch 76/500
17239/17239 [==============================] - 0s - loss: 0.6814 - acc: 0.5431 - recall: 0.5876     
Epoch 77/500
17239/17239 [==============================] - 0s - loss: 0.6812 - acc: 0.5741 - recall: 0.7932     
Epoch 78/500
17239/17239 [==============================] - 0s - loss: 0.6801 - acc: 0.5768 - recall: 0.7841     
Epoch 79/500
17239/17239 [==============================] - 0s - loss: 0.6770 - acc: 0.5732 - recall: 0.7385     
Epoch 80/500
17239/17239 [==============================] - 0s - loss: 0.6750 - acc: 0.5735 - recall:

17239/17239 [==============================] - 0s - loss: 0.6186 - acc: 0.6394 - recall: 0.7324     
Epoch 144/500
17239/17239 [==============================] - 0s - loss: 0.6168 - acc: 0.6432 - recall: 0.7487     
Epoch 145/500
17239/17239 [==============================] - 0s - loss: 0.6159 - acc: 0.6452 - recall: 0.7410     
Epoch 146/500
17239/17239 [==============================] - 0s - loss: 0.6160 - acc: 0.6422 - recall: 0.7423     
Epoch 147/500
17239/17239 [==============================] - 0s - loss: 0.6160 - acc: 0.6398 - recall: 0.7279     
Epoch 148/500
17239/17239 [==============================] - 0s - loss: 0.6161 - acc: 0.6401 - recall: 0.7270     
Epoch 149/500
17239/17239 [==============================] - 0s - loss: 0.6152 - acc: 0.6448 - recall: 0.7425     
Epoch 150/500
17239/17239 [==============================] - 0s - loss: 0.6156 - acc: 0.6428 - recall: 0.7329     
Epoch 151/500
17239/17239 [==============================] - 0s - loss: 0.6147 - acc: 0.6426 -

17239/17239 [==============================] - 0s - loss: 0.5979 - acc: 0.6515 - recall: 0.7098     
Epoch 215/500
17239/17239 [==============================] - 0s - loss: 0.5974 - acc: 0.6520 - recall: 0.7099     
Epoch 216/500
17239/17239 [==============================] - 0s - loss: 0.5987 - acc: 0.6522 - recall: 0.7101     
Epoch 217/500
17239/17239 [==============================] - 0s - loss: 0.6002 - acc: 0.6511 - recall: 0.7122     
Epoch 218/500
17239/17239 [==============================] - 0s - loss: 0.5992 - acc: 0.6497 - recall: 0.7192     
Epoch 219/500
17239/17239 [==============================] - 0s - loss: 0.6007 - acc: 0.6496 - recall: 0.7136     
Epoch 220/500
17239/17239 [==============================] - 0s - loss: 0.6010 - acc: 0.6513 - recall: 0.6988     
Epoch 221/500
17239/17239 [==============================] - 0s - loss: 0.5995 - acc: 0.6501 - recall: 0.7075     
Epoch 222/500
17239/17239 [==============================] - 0s - loss: 0.5969 - acc: 0.6529 -

17239/17239 [==============================] - 0s - loss: 0.6201 - acc: 0.6339 - recall: 0.5980     
Epoch 286/500
17239/17239 [==============================] - 0s - loss: 0.6848 - acc: 0.5771 - recall: 0.3477     
Epoch 287/500
17239/17239 [==============================] - 0s - loss: 0.6578 - acc: 0.5817 - recall: 0.3850     
Epoch 288/500
17239/17239 [==============================] - 0s - loss: 0.6463 - acc: 0.5966 - recall: 0.4449     
Epoch 289/500
17239/17239 [==============================] - 0s - loss: 0.6407 - acc: 0.6090 - recall: 0.5560     
Epoch 290/500
17239/17239 [==============================] - 0s - loss: 0.6334 - acc: 0.6045 - recall: 0.5504     
Epoch 291/500
17239/17239 [==============================] - 0s - loss: 0.6285 - acc: 0.6032 - recall: 0.5792     
Epoch 292/500
17239/17239 [==============================] - 0s - loss: 0.6255 - acc: 0.6124 - recall: 0.6547     
Epoch 293/500
17239/17239 [==============================] - 0s - loss: 0.6286 - acc: 0.6061 -

17239/17239 [==============================] - 0s - loss: 0.5885 - acc: 0.6583 - recall: 0.7212     
Epoch 357/500
17239/17239 [==============================] - 0s - loss: 0.5885 - acc: 0.6571 - recall: 0.7172     
Epoch 358/500
17239/17239 [==============================] - 0s - loss: 0.5904 - acc: 0.6586 - recall: 0.7156     
Epoch 359/500
17239/17239 [==============================] - 0s - loss: 0.5900 - acc: 0.6583 - recall: 0.7197     
Epoch 360/500
17239/17239 [==============================] - 0s - loss: 0.5881 - acc: 0.6560 - recall: 0.7132     
Epoch 361/500
17239/17239 [==============================] - 0s - loss: 0.5924 - acc: 0.6578 - recall: 0.7047     
Epoch 362/500
17239/17239 [==============================] - 0s - loss: 0.5930 - acc: 0.6531 - recall: 0.6816     
Epoch 363/500
17239/17239 [==============================] - 0s - loss: 0.6067 - acc: 0.6498 - recall: 0.6801     
Epoch 364/500
17239/17239 [==============================] - 0s - loss: 0.5970 - acc: 0.6535 -

17239/17239 [==============================] - 0s - loss: 0.5948 - acc: 0.6569 - recall: 0.7122     
Epoch 428/500
17239/17239 [==============================] - 0s - loss: 0.5999 - acc: 0.6486 - recall: 0.7027     
Epoch 429/500
17239/17239 [==============================] - 0s - loss: 0.5951 - acc: 0.6576 - recall: 0.7157     
Epoch 430/500
17239/17239 [==============================] - 0s - loss: 0.5910 - acc: 0.6555 - recall: 0.7169     
Epoch 431/500
17239/17239 [==============================] - 0s - loss: 0.5881 - acc: 0.6591 - recall: 0.7192     
Epoch 432/500
17239/17239 [==============================] - 0s - loss: 0.5895 - acc: 0.6586 - recall: 0.7092     
Epoch 433/500
17239/17239 [==============================] - 0s - loss: 0.5881 - acc: 0.6588 - recall: 0.7093     
Epoch 434/500
17239/17239 [==============================] - 0s - loss: 0.5880 - acc: 0.6593 - recall: 0.7130     
Epoch 435/500
17239/17239 [==============================] - 0s - loss: 0.5873 - acc: 0.6590 -

17239/17239 [==============================] - 0s - loss: 0.5913 - acc: 0.6580 - recall: 0.7096     
Epoch 499/500
17239/17239 [==============================] - 0s - loss: 0.5899 - acc: 0.6568 - recall: 0.7067     
Epoch 500/500
17239/17239 [==============================] - 0s - loss: 0.5887 - acc: 0.6592 - recall: 0.7151     

acc: 65.95%
